In [2]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain_openai import ChatOpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()

# Initialize LLM - will use OPENAI_API_KEY from environment
llm = ChatOpenAI(model="gpt-4o-mini")

# Step 1: Summarize
summary_prompt = PromptTemplate(
    input_variables=["report"],
    template="Summarize the key findings of the following market research report:\n{report}"
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")

# Step 2: Identify Trends (JSON format) // an example of structured output
trend_prompt = PromptTemplate(
    input_variables=["summary"],
    template="""From the following summary, extract exactly 3 emerging trends. 
Return ONLY a valid JSON array with this exact structure:
[
  {{
    "trend": "trend name",
    "supporting_data": ["data point 1", "data point 2"]
  }},
  {{
    "trend": "trend name",
    "supporting_data": ["data point 1", "data point 2"]
  }},
  {{
    "trend": "trend name",
    "supporting_data": ["data point 1", "data point 2"]
  }}
]

Summary:
{summary}

Return ONLY the JSON array, no additional text or explanation:"""
)
trend_chain = LLMChain(llm=llm, prompt=trend_prompt, output_key="trends")

# Step 3: Draft Email
email_prompt = PromptTemplate(
    input_variables=["trends"],
    template="Draft a concise professional email summarizing these trends:\n{trends}"
)
email_chain = LLMChain(llm=llm, prompt=email_prompt, output_key="email")

# Combine into one pipeline
pipeline = SequentialChain(
    chains=[summary_chain, trend_chain, email_chain],
    input_variables=["report"],
    output_variables=["summary", "trends", "email"],
    verbose=True
)

# Run the pipeline, "market_reports.txt" is a file that contains the market research report, replace with your own file
report_text = open("market_reports.txt", encoding="utf-8").read()
final_output = pipeline.invoke({"report": report_text})

# Beautify output
print("\n" + "="*70)
print("EXECUTIVE SUMMARY")
print("="*70)
print(final_output["summary"])

print("\n" + "="*70)
print("EMERGING TRENDS")
print("="*70)
print(final_output["trends"])

print("\n" + "="*70)
print("PROFESSIONAL EMAIL")
print("="*70)
print(final_output["email"])


C:\Users\varsh\AppData\Local\Temp\ipykernel_19112\2333492079.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")




> Entering new SequentialChain chain...

> Finished chain.

EXECUTIVE SUMMARY
The "2025 India Retail & E-commerce Market — Q1 Insights" report reveals several key findings:

1. **Market Growth**: Overall retail GMV grew by 11.8% year-over-year (YoY), with e-commerce seeing a more robust increase of 18.6%, largely fueled by the penetration in Tier-2 and Tier-3 cities.

2. **AI Adoption**: The adoption of AI among mid-to-large retailers rose significantly from 27% to 41%. The most common applications of AI include search and personalization (63%), demand forecasting (52%), and customer support (47%).

3. **E-commerce Payment Trends**: Unified Payments Interface (UPI) has become the dominant payment method for e-commerce, accounting for 64% of transactions, with the average order value (AOV) rising to ₹1,420, representing a 6% YoY increase.

4. **Logistics Improvements**: There was a notable enhancement in logistics, with same-day and next-day delivery coverage increasing from 22% to 35